# Mixed Momentum Strategy

In [1]:
import pandas as pd
import numpy as np

from warnings import filterwarnings
filterwarnings('ignore')

path_intermediate = '../intermediate_yilin/'

In [2]:
signals_master = pd.read_csv(path_intermediate + 'SignalMasterTable.csv', index_col=['Unnamed: 0'], parse_dates=['time_avail_m'])
signals_master

,permno,ret,prc,shrcd,exchcd,sicCRSP,ticker,time_avail_m,mve_c,gvkey,sicCS,tickerIBES,NYSE,bh1m
0,10000.0,NaN,-4.375000,10.0,3.0,3990.0,OMFGA,1986-01-01,1.610000e+01,NaN,NaN,NaN,0.0,-0.257143
1,10000.0,-0.257143,-3.250000,10.0,3.0,3990.0,OMFGA,1986-02-01,1.196000e+01,NaN,NaN,NaN,0.0,0.365385
2,10000.0,0.365385,-4.437500,10.0,3.0,3990.0,OMFGA,1986-03-01,1.633000e+01,NaN,NaN,NaN,0.0,-0.098592
3,10000.0,-0.098592,-4.000000,10.0,3.0,3990.0,OMFGA,1986-04-01,1.517200e+01,NaN,NaN,NaN,0.0,-0.222656
4,10000.0,-0.222656,-3.109375,10.0,3.0,3990.0,OMFGA,1986-05-01,1.179386e+01,NaN,NaN,NaN,0.0,-0.005025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3879455,93436.0,0.027612,1144.760010,11.0,3.0,9999.0,TSLA,2021-11-01,1.149642e+06,184996.0,3711.0,TSLA,0.0,-0.076855
3879456,93436.0,-0.076855,1056.780029,11.0,3.0,9999.0,TSLA,2021-12-01,1.092218e+06,184996.0,3711.0,TSLA,0.0,-0.113609
3879457,93436.0,-0.113609,936.719971,11.0,3.0,9999.0,TSLA,2022-01-01,9.681319e+05,184996.0,3711.0,TSLA,0.0,-0.070768
3879458,93436.0,-0.070768,870.429993,11.0,3.0,9999.0,TSLA,2022-02-01,8.996190e+05,184996.0,3711.0,TSLA,0.0,0.238009


## Firm Age Mom

In [3]:
strat_df = signals_master.copy()

# Counting Firm age by number of PERNOs per day
strat_df.sort_values(['permno','time_avail_m'], inplace=True)
strat_df['tempage'] = strat_df.groupby(['permno']).cumcount()+1

In [5]:
strat_df['ret'] = np.log(1 + strat_df['ret'])

In [10]:
strat_df

,permno,ret,prc,shrcd,exchcd,sicCRSP,ticker,time_avail_m,mve_c,gvkey,sicCS,tickerIBES,NYSE,bh1m,tempage
0,10000.0,NaN,-4.375000,10.0,3.0,3990.0,OMFGA,1986-01-01,1.610000e+01,NaN,NaN,NaN,0.0,-0.257143,1
1,10000.0,-0.297252,-3.250000,10.0,3.0,3990.0,OMFGA,1986-02-01,1.196000e+01,NaN,NaN,NaN,0.0,0.365385,2
2,10000.0,0.311436,-4.437500,10.0,3.0,3990.0,OMFGA,1986-03-01,1.633000e+01,NaN,NaN,NaN,0.0,-0.098592,3
3,10000.0,-0.103797,-4.000000,10.0,3.0,3990.0,OMFGA,1986-04-01,1.517200e+01,NaN,NaN,NaN,0.0,-0.222656,4
4,10000.0,-0.251873,-3.109375,10.0,3.0,3990.0,OMFGA,1986-05-01,1.179386e+01,NaN,NaN,NaN,0.0,-0.005025,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3879455,93436.0,0.027238,1144.760010,11.0,3.0,9999.0,TSLA,2021-11-01,1.149642e+06,184996.0,3711.0,TSLA,0.0,-0.076855,138
3879456,93436.0,-0.079968,1056.780029,11.0,3.0,9999.0,TSLA,2021-12-01,1.092218e+06,184996.0,3711.0,TSLA,0.0,-0.113609,139
3879457,93436.0,-0.120597,936.719971,11.0,3.0,9999.0,TSLA,2022-01-01,9.681319e+05,184996.0,3711.0,TSLA,0.0,-0.070768,140
3879458,93436.0,-0.073397,870.429993,11.0,3.0,9999.0,TSLA,2022-02-01,8.996190e+05,184996.0,3711.0,TSLA,0.0,0.238009,141


In [11]:
# Computing Cumulative Returns over past period for Momentum 
strat_df['firm_age_mom'] = strat_df.groupby('permno')['ret'].rolling(window=6).sum().reset_index(level=0, drop=True)

In [23]:
# Quantile Transform for momentum 
strat_df['mom_high'] = strat_df['ret'] > strat_df.groupby('time_avail_m')['ret'].transform('quantile', 0.8)
strat_df['mom_low'] = strat_df['ret'] < strat_df.groupby('time_avail_m')['ret'].transform('quantile', 0.2)

In [24]:
# Quantile Transform for age 
strat_df['age_low'] = strat_df['tempage'] < strat_df.groupby('time_avail_m')['tempage'].transform('quantile', 0.2)
strat_df['age_high'] = strat_df['tempage'] > strat_df.groupby('time_avail_m')['tempage'].transform('quantile', 0.8)

In [25]:
# Some basic Filtering 
strat_df['firm_age_mom'].loc[(np.abs(strat_df['prc'])<5) | (strat_df['tempage']<12)] = np.nan

In [26]:
strat_df

,permno,ret,prc,shrcd,exchcd,sicCRSP,ticker,time_avail_m,mve_c,gvkey,sicCS,tickerIBES,NYSE,bh1m,tempage,firm_age_mom,mom_high,mom_low,age_low,age_high
0,10000.0,NaN,-4.375000,10.0,3.0,3990.0,OMFGA,1986-01-01,1.610000e+01,NaN,NaN,NaN,0.0,-0.257143,1,NaN,False,False,True,False
1,10000.0,-0.297252,-3.250000,10.0,3.0,3990.0,OMFGA,1986-02-01,1.196000e+01,NaN,NaN,NaN,0.0,0.365385,2,NaN,False,True,True,False
2,10000.0,0.311436,-4.437500,10.0,3.0,3990.0,OMFGA,1986-03-01,1.633000e+01,NaN,NaN,NaN,0.0,-0.098592,3,NaN,True,False,True,False
3,10000.0,-0.103797,-4.000000,10.0,3.0,3990.0,OMFGA,1986-04-01,1.517200e+01,NaN,NaN,NaN,0.0,-0.222656,4,NaN,False,True,True,False
4,10000.0,-0.251873,-3.109375,10.0,3.0,3990.0,OMFGA,1986-05-01,1.179386e+01,NaN,NaN,NaN,0.0,-0.005025,5,NaN,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3879455,93436.0,0.027238,1144.760010,11.0,3.0,9999.0,TSLA,2021-11-01,1.149642e+06,184996.0,3711.0,TSLA,0.0,-0.076855,138,0.604847,True,False,False,False
3879456,93436.0,-0.079968,1056.780029,11.0,3.0,9999.0,TSLA,2021-12-01,1.092218e+06,184996.0,3711.0,TSLA,0.0,-0.113609,139,0.441330,False,False,False,False
3879457,93436.0,-0.120597,936.719971,11.0,3.0,9999.0,TSLA,2022-01-01,9.681319e+05,184996.0,3711.0,TSLA,0.0,-0.070768,140,0.309759,False,False,False,False
3879458,93436.0,-0.073397,870.429993,11.0,3.0,9999.0,TSLA,2022-02-01,8.996190e+05,184996.0,3711.0,TSLA,0.0,0.238009,141,0.168138,False,False,False,False


## Announcement Return

In [135]:
earnings = pd.read_csv('./earnings.csv')

In [136]:
earnings = earnings.dropna(subset=['rdq', 'epsfxq'])

In [137]:
earnings

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,curcdq,datacqtr,datafqtr,rdq,epsfxq,costat
24,1000,1972-03-31,1972,1.0,INDL,C,D,STD,USD,1972Q1,1972Q1,1972-04-20,0.10,I
25,1000,1972-06-30,1972,2.0,INDL,C,D,STD,USD,1972Q2,1972Q2,1972-07-24,0.17,I
26,1000,1972-09-30,1972,3.0,INDL,C,D,STD,USD,1972Q3,1972Q3,1972-10-17,0.13,I
27,1000,1972-12-31,1972,4.0,INDL,C,D,STD,USD,1972Q4,1972Q4,1973-02-15,0.12,I
28,1000,1973-03-31,1973,1.0,INDL,C,D,STD,USD,1973Q1,1973Q1,1973-04-23,0.13,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1965088,353444,2022-06-30,2022,2.0,INDL,C,D,STD,USD,2022Q2,2022Q2,2022-09-20,0.04,A
1965089,353444,2022-09-30,2022,3.0,INDL,C,D,STD,USD,2022Q3,2022Q3,2022-11-10,0.07,A
1965090,353444,2022-12-31,2022,4.0,INDL,C,D,STD,USD,2022Q4,2022Q4,2023-03-02,0.07,A
1965091,353444,2023-03-31,2023,1.0,INDL,C,D,STD,USD,2023Q1,2023Q1,2023-05-03,0.10,A


In [138]:
earnings['diff_eps'] = earnings.groupby('gvkey')['epsfxq'].diff(3)

In [139]:
earnings = earnings.dropna(subset='diff_eps')

In [140]:
earnings['std_diff_eps'] = earnings['diff_eps'].rolling(3).std()

In [141]:
earnings['sue'] = earnings['diff_eps'] / earnings['std_diff_eps']

In [142]:
earnings = earnings.dropna(subset='sue')

In [143]:
earnings['datadate'] = pd.to_datetime(earnings['datadate']) + pd.DateOffset(days=1)

In [144]:
earnings = earnings.rename(columns ={'datadate': 'time_avail_m'})

In [151]:
earnings.index = earnings['time_avail_m']

In [153]:
sue_df = earnings.groupby('gvkey')['sue'].resample('MS').first().ffill().reset_index()

In [155]:
sue_df

,gvkey,time_avail_m,sue
0,1000,1973-07-01,1.091089
1,1000,1973-08-01,1.091089
2,1000,1973-09-01,1.091089
3,1000,1973-10-01,-1.023984
4,1000,1973-11-01,-1.023984
...,...,...,...
3582730,353444,2022-12-01,-0.250619
3582731,353444,2023-01-01,-0.126366
3582732,353444,2023-02-01,-0.126366
3582733,353444,2023-03-01,-0.126366


In [156]:
lo = pd.merge(strat_df, sue_df, on=['gvkey', 'time_avail_m'])

In [157]:
lo

,permno,ret,prc,shrcd,exchcd,sicCRSP,ticker,time_avail_m,mve_c,gvkey,...,tickerIBES,NYSE,bh1m,tempage,firm_age_mom,mom_high,mom_low,age_low,age_high,sue
0,10001.0,0.021053,6.000000,11.0,3.0,4920.0,GFGC,1987-07-01,5.946000e+00,12994.0,...,GFGC,0.0,0.083333,19,-0.083732,False,False,False,False,0.693870
1,10001.0,0.080043,6.500000,11.0,3.0,4920.0,GFGC,1987-08-01,6.441500e+00,12994.0,...,GFGC,0.0,-0.022308,20,0.073272,False,False,False,False,0.693870
2,10001.0,-0.022560,6.250000,11.0,3.0,4920.0,GFGC,1987-09-01,6.200000e+00,12994.0,...,GFGC,0.0,0.020000,21,0.014572,False,False,False,False,0.693870
3,10001.0,0.019803,6.375000,11.0,3.0,4920.0,GFGC,1987-10-01,6.324000e+00,12994.0,...,GFGC,0.0,-0.029412,22,0.074380,True,False,False,False,-1.380985
4,10001.0,-0.029853,6.187500,11.0,3.0,4920.0,GFGC,1987-11-01,6.138000e+00,12994.0,...,GFGC,0.0,-0.033535,23,0.118635,False,False,False,False,-1.380985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2298270,93436.0,0.027238,1144.760010,11.0,3.0,9999.0,TSLA,2021-11-01,1.149642e+06,184996.0,...,TSLA,0.0,-0.076855,138,0.604847,True,False,False,False,1.809697
2298271,93436.0,-0.079968,1056.780029,11.0,3.0,9999.0,TSLA,2021-12-01,1.092218e+06,184996.0,...,TSLA,0.0,-0.113609,139,0.441330,False,False,False,False,1.809697
2298272,93436.0,-0.120597,936.719971,11.0,3.0,9999.0,TSLA,2022-01-01,9.681319e+05,184996.0,...,TSLA,0.0,-0.070768,140,0.309759,False,False,False,False,3.678058
2298273,93436.0,-0.073397,870.429993,11.0,3.0,9999.0,TSLA,2022-02-01,8.996190e+05,184996.0,...,TSLA,0.0,0.238009,141,0.168138,False,False,False,False,3.678058


In [ ]:
lo['sue_high'] = 